In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# LCLS Classic model 

In [ ]:
from lcls_live.bmad import LCLSTaoModel
from lcls_live.epics import epics_proxy

import numpy as np
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Make sure this exists
assert 'LCLS_LATTICE' in os.environ

In [ ]:
# Cached EPICS pv data
epics = epics_proxy('data/epics_snapshot_2018-03-06T11:22:45.000000-08:00.json', verbose=True)

In [ ]:
M = LCLSTaoModel('cu_sxr', epics = epics,verbose=True, ploton=True)
print(M)

In [ ]:
%%tao 
sho var linac_fudge

## Track particles

In [ ]:
%%tao
set beam_init beam_track_end = UEEND
set csr_param n_bin = 40
snparticle 10000
beamon
beamoff

In [ ]:
h5file1 = os.path.join(os.getcwd(), 'test1.h5')
M.cmd('write beam -at UEBEG '+h5file1)

h5file2 = os.path.join(os.getcwd(), 'test2.h5')
M.cmd('write beam -at UEEND '+h5file2)

# Plot

In [ ]:
from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
from h5py import File

In [ ]:
#h1.close()
#h2.close()
h1 = File(h5file1, 'r')
h2 = File(h5file2, 'r')
bunch1 = h1['data/00001/particles']
bunch2 = h2['data/00001/particles']

show(
    row(
        column(    
            bunch_plotting.plot_bunch_h5(bunch1, 't', 'pz', liveOnly=True,  bins = 100),
            bunch_plotting.plot_histogram_h5(bunch1, 't', bins=100)), 
    column(
        
    bunch_plotting.plot_bunch_h5(bunch2, 't', 'pz', liveOnly=True,  bins = 100),
    bunch_plotting.plot_histogram_h5(bunch2, 't', bins=100))        

    )
    
    )

In [ ]:
# Cleanup
os.remove(h5file1), os.remove(h5file2)